In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.documents import Document
from langchain_community.document_loaders import JSONLoader
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from urllib.parse import urlparse

import os
import chromadb

e:\case\AI-hackaton-kemenkes\agent\ragagent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
if not os.getenv("GEMINI_API_KEY") and not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GEMINI_API_KEY atau GOOGLE_API_KEY tidak ditemukan di environment variables.")

chromaDB = os.getenv("CHROMA_DB_URL")
parsed_url = urlparse(chromaDB)
host = parsed_url.hostname
port = parsed_url.port

client = chromadb.HttpClient(host=host, port=port)

In [3]:
EMBEDDING_MODEL = GoogleGenerativeAIEmbeddings(model="text-embedding-004")
LLM_MODEL = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)

In [4]:
class DataProcessor:
    def __init__(self, chunk_size: int = 1000, overlap: int = 200):
        self.textSplitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=overlap,
            separators=["\n\n", "\n", ".", " "]
        )

    def processText(self, text: str) -> list[Document]:
        """ Memecah teks panjang menjadi chunks dan konversi ke objek Document """
        chunks = self.textSplitter.split_text(text)
        return [Document(page_content=chunk) for chunk in chunks]

In [5]:
class VectorDatabase:
    def __init__(self, embeddingFunction : EMBEDDING_MODEL, vectorStore : Chroma, client : client):
        self.embeddingFunction = embeddingFunction
        self.vectorStore = vectorStore
        self.client = client
    
    def createIndex(self, documents: list[Document]):
        """ Membuat indeks vector store menggunakan FAISS dengan wrapper LangChain """

        self.vectorStore = self.vectorStore.get_or_create_collection("gizi_data")

        for doc in documents:
            content = doc.page_content if hasattr(doc, 'page_content') else doc['content']

            embedding = self.embeddingFunction.embed_documents([content])[0]
            
            self.vectorStore.add(
                [embedding],  # 
                metadatas=[{
                    "id": doc.get("id", "N/A"),
                    "topic": doc.get("topic", "N/A"),
                    "sub_topic": doc.get("sub_topic", "N/A"),
                    "source": doc.get("source", "N/A"),
                    "date_updated": doc.get("date_updated", "N/A"),
                }],
                ids=[f"doc_{doc['id']}"]  
            )
            
        return self.vectorStore

In [6]:
class Retriever:
    def __init__(self, vectorStore : Chroma):
        self.retriever = vectorStore.as_retriever(search_kwargs={"k": 5})
        self.vectorStore = vectorStore
    
    def retrieve(self, query: str) -> list[Document]:
        """ Melakukan pencarian pada vector store untuk menemukan data relevan """
        return self.retriever.invoke(query)

In [7]:
class ChatbotAgent:
    def __init__(self, vectorStore, llm_model: ChatGoogleGenerativeAI):
        self.vectorStore = vectorStore
        self.llm = llm_model
        self.retriever = Retriever(vectorStore)
    
        # PROMPT RAG
        self.promptTemplate = ChatPromptTemplate.from_messages([
            ("system", """
                Anda adalah Asisten Pakar Gizi dan Pencegahan stunting untuk ibu hamil bernama MateBot panggil setiap user Bunda. Jawablah pertanyaan pengguna **HANYA** berdasarkan konteks yang diberikan di bawah.
                Pastikan jawaban Anda:
                1. Menggunakan bahasa Indonesia formal, ramah, dan informatif.
                2. Menyebutkan **Definisi**, **Penyebab**, dan **Pencegahan** jika relevan.
                3. Gunakan bullet point atau penomoran untuk memudahkan pembacaan.

                KONTEKS:
                {context}
                """),
                ("user", "{query}")
        ])

        self.FALLBACK_PROMPT = ChatPromptTemplate.from_messages([
            ("system", """
                Bunda bertanya tentang '{query}'. Informasi spesifik tidak ditemukan di basis data gizi MateBot.
                Jawab pertanyaan ini menggunakan pengetahuan umum Anda HANYA JIKA topik tersebut masih berhubungan erat dengan Gizi, Kehamilan, atau Stunting. 
                # Jika pertanyaan sama sekali tidak berhubungan dengan topik ini, jawab: 'Maaf Bunda, saya hanya dilatih untuk memberikan informasi spesifik mengenai gizi dan pencegahan stunting.' 
                Berikan jawaban dengan memanggil user 'Bunda' dan berikan disclaimer bahwa ini adalah informasi umum.
            """),
            ("user", "{query}")
        ])
    
    def generateResponse(self, query: str):
        """ Menghasilkan respons menggunakan retriever dan model generatif (Gemini) """
        
        DOC_COUNT = 5 
        DISTANCE_THRESHOLD = 0.4 

        scoredDocs = self.vectorStore.similarity_search_with_score(query, k=DOC_COUNT)
        
        bestDistance = scoredDocs[0][1] if scoredDocs else 999.0
        
        retrievedData = []
        
        if bestDistance > DISTANCE_THRESHOLD:
            response = self.llm.invoke(self.FALLBACK_PROMPT.format(query=query))
            retrievedData.append(Document(page_content="[SUMBER: Pengetahuan Umum MateBot (Tidak bersumber dari data gizi spesifik).]", metadata={"source": "Gemini Knowledge"}))
            
        else:
            retrievedData = self.retriever.retrieve(query)
            context = "\n---\n".join([doc.page_content for doc in retrievedData])
            
            response = self.llm.invoke(
                self.promptTemplate.format_messages(context=context, query=query)
            )

        return {
            "answer": response.content,
            "source_documents": retrievedData
        }

### ==============================================================================
### GLOBAL FUNCTIONS
### ==============================================================================

In [8]:
def processKnowledgeData(file_path: str = "../data/giziData.json"):
    """ Alur kerja Indexing: Load, Split, Embed, Store """
    loader = JSONLoader(
        file_path=file_path,
        jq_schema='.[]', 
        text_content=False, 
        content_key="content",
        
        metadata_func=lambda record, metadata: {
            "id": record.get("id", "N/A"),
            "topic": record.get("topic", "N/A"),
            "sub_topic": record.get("sub_topic", "N/A"),
            "source": record.get("source", "N/A"),
            "date_updated": record.get("date_updated", "N/A"),
        }
    )

    documents = loader.load()

    dataProcessor = DataProcessor() 
    documentsAfterSplit = dataProcessor.textSplitter.split_documents(documents)

    
    vectorStore = Chroma.from_documents(
        documents=documentsAfterSplit, 
        embedding=EMBEDDING_MODEL, 
        client=client,
        collection_name="gizi_data"
    )
    
    return vectorStore

In [9]:
try:
    collection_native = client.get_collection("gizi_data")
    print(client)
    VECTOR_STORE_GLOBAL = Chroma(
        client=client,
        collection_name="gizi_data",
        embedding_function=EMBEDDING_MODEL 
    )
    print("✅ Vector Store berhasil dimuat dari Chroma DB.")
except:
    print("⚠️ Vector Store tidak ditemukan di Chroma DB, membuat indeks baru...")
    VECTOR_STORE_GLOBAL = processKnowledgeData()

✅ Vector Store berhasil dimuat dari Chroma DB.


C:\Users\aafau\AppData\Local\Temp\ipykernel_26456\2868910057.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  VECTOR_STORE_GLOBAL = Chroma(


In [ ]:
def mainFlow(query):
    """ Fungsi utama untuk menjalankan RAG """
    chatbot = ChatbotAgent(
        vectorStore=VECTOR_STORE_GLOBAL, 
        llm_model=LLM_MODEL
    ) 
    response = chatbot.generateResponse(query) 
    return response


In [11]:
query = "Apa itu stunting"
response = mainFlow(query)

print("--- Hasil Jawaban Chatbot ---")
print(response['answer'])

print("\n--- Dokumen Sumber yang Digunakan ---")
for doc in response['source_documents']:

    print(f"- {doc.page_content[:150]}...")

--- Hasil Jawaban Chatbot ---
Halo, Bunda! Saya MateBot, siap membantu Bunda.

Bunda bertanya mengenai apa itu stunting. Berdasarkan informasi yang ada, berikut adalah **Definisi** stunting:

*   Stunting adalah gangguan pertumbuhan dan perkembangan anak balita (di bawah lima tahun) yang ditandai dengan tinggi badan lebih rendah dari standar usianya.
*   Kondisi ini disebabkan oleh kekurangan gizi kronis dan infeksi berulang, terutama selama periode 1.000 Hari Pertama Kehidupan (HPK).
*   Dampak stunting tidak hanya membuat anak pendek, tetapi juga dapat menghambat perkembangan kognitif, meningkatkan risiko penyakit, dan menurunkan produktivitas di masa depan.

--- Dokumen Sumber yang Digunakan ---
- Penyebab stunting dari segi gizi meliputi: Kekurangan kalori yang menghambat pertumbuhan fisik; Kekurangan protein yang menyebabkan keterlambatan perk...
- Penyebab stunting dari segi gizi meliputi: Kekurangan kalori yang menghambat pertumbuhan fisik; Kekurangan protein yang menyebabkan ke